In [1]:
from flows.candidate_link import find_candidate_amino_atom
from flows.post_process import build_la_ra_mapping_by_idx
from flows.utils import save_results
from objects.data_builder import DataBuilder
from models.configs import model_configs
from models import create_model
from utils import *
import pandas as pd
import configs
import time
import sys
import os

In [2]:
DataBuilder().init(configs.data_path)
list_receptors, list_ligands = DataBuilder().get_atom_data()

In [3]:
list_ligands[0].ligand_df.df['ATOM']

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,9,,N,,ALA,,C,2,,...,-12.583,6.625,72.374,1.0,41.65,,,N,NaN,11
1,ATOM,10,,CA,,ALA,,C,2,,...,-13.166,7.625,71.495,1.0,42.30,,,C,NaN,12
2,ATOM,11,,C,,ALA,,C,2,,...,-12.115,8.750,71.241,1.0,41.96,,,C,NaN,13
3,ATOM,12,,O,,ALA,,C,2,,...,-11.466,9.159,72.125,1.0,41.95,,,O,NaN,14
4,ATOM,13,,CB,,ALA,,C,2,,...,-14.397,8.218,72.132,1.0,36.98,,,C,NaN,15
5,ATOM,14,,N,,VAL,,C,3,,...,-11.943,9.281,69.911,1.0,34.91,,,N,NaN,16
6,ATOM,15,,CA,,VAL,,C,3,,...,-10.955,10.336,69.666,1.0,36.29,,,C,NaN,17
7,ATOM,16,,C,,VAL,,C,3,,...,-11.663,11.606,69.273,1.0,40.78,,,C,NaN,18
8,ATOM,17,,O,,VAL,,C,3,,...,-12.381,11.617,68.328,1.0,36.69,,,O,NaN,19
9,ATOM,18,,CB,,VAL,,C,3,,...,-10.004,9.895,68.515,1.0,37.53,,,C,NaN,20


In [4]:
list_receptors[0]

In [5]:
list_receptors[0].receptor_atoms[list_receptors[0].receptor_atoms["atom_number"] == 2].index.to_numpy()[0]

1

In [1]:
from rdkit.Chem.rdmolfiles import *
from rdkit.Chem.rdchem import *
from rdkit.Chem.rdmolops import *
from rdkit.Chem.Draw import *
from rdkit.Chem.Lipinski import *

# Setting for auto draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.molSize = 1000,1000

In [12]:
import glob
import os

list_folder = glob.glob('Mpro/aligned/*')
for lf in list_folder:
    list_file = glob.glob('{}/*'.format(lf))
    name = lf.split('/')[-1]
    not_del_file = [name+'.'+ex for ex in ['mol','pdb']]
    for f in list_file:
        if f.split('/')[-1] not in not_del_file:
            if os.path.exists(f):
                try:
                    os.remove(f)
                except:
                    print('Not found')
    


In [10]:
from openbabel import openbabel

obConversion = openbabel.OBConversion()
obConversion.SetInAndOutFormats("pdbqt", "pdb")



True

In [22]:
import glob
path = 'data/receptors/'
list_files = glob.glob('/Users/duckhoan/Documents/VS_Code/ViDok/Auto_download_vidok/Ligand_receptor/*_receptor.pdbqt')
for lf in list_files:
    name = '_'.join(lf.split('/')[-1].split('_')[:-1])
    try:
        obConversion.OpenInAndOutFiles(lf,path+name+'.pdb')
        obConversion.Convert()
        obConversion.CloseOutFile()
    except:
        print(lf) 

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/duckhoan/Documents/VS_Code/ViDok/Auto_download_vidok/Ligand_receptor/ZINC000085862539_929_receptor.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/duckhoan/Documents/VS_Code/ViDok/Auto_download_vidok/Ligand_receptor/ZINC000085862539_1050_receptor.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/duckhoan/Documents/VS_Code/ViDok/Auto_download_vidok/Ligand_receptor/ZINC000085862539_1045_receptor.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/duckhoan/Documents/VS_Code/ViDok/Auto_download_vidok/Ligand_receptor/ZINC000085862539_500_receptor.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to 

In [23]:
print(len(glob.glob('results/hydrogen_bonds/hydrogen_bonds/*.csv')))

911


In [19]:
import shutil
import glob

list_files = glob.glob('/Users/duckhoan/Documents/VS_Code/ViDok/GNN_DTI/Vidok_Crawler/Data_Vidok_Infer/*_pocket.pdb')
for f in list_files:
    shutil.copy(f, 'data/receptors')

In [25]:
import filecmp
# list_files = glob.glob('')
print(filecmp.cmp('data/receptors/ZINC000000150863_2.pdb', 'data/receptors/ZINC000012153879_16.pdb'))


False


In [21]:
import os
import glob
list_files = glob.glob('data/receptors/*.pdb')
for f in list_files:
    name = '_'.join(f.split('/')[-1].split('_')[:-1])
    try:
        os.rename(f, 'data/receptors/{}.pdb'.format(name))
    except:
        print(name)

In [13]:
from openbabel import openbabel

obConversion = openbabel.OBConversion()
obConversion.SetInAndOutFormats("mol", "mol2")

obConversion.OpenInAndOutFiles("data/Mpro-P0008_0A.mol","data/Mpro-P0008_0A.mol2")
obConversion.Convert()
obConversion.CloseOutFile()

# obConversion.OpenInAndOutFiles("data/receptors/Receptor_ViDok.pdb","data/receptors/SDF/Receptor_ViDok.sdf")
# obConversion.Convert()
# obConversion.CloseOutFile()

In [14]:
import glob
import os
import shutil
# list_file = glob.glob("/Users/duckhoan/Documents/VS_Code/ViDok/Auto_download_vidok/complex_ligand/*.sdf")
# name_sdf = [path.split(".")[0] for path in list_file]
# print(len(name_sdf))
# list_file_pdb = glob.glob("/Users/duckhoan/Documents/VS_Code/ViDok/Auto_download_vidok/complex_ligand/*.pdbqt")
# name_pdb = [path.split(".")[0] for path in list_file_pdb]
# print(len(name_pdb))

list_sdf_after = glob.glob("/Users/duckhoan/Documents/VS_Code/ViDok/Auto_download_vidok/ligand_after_format/*.sdf")
name_sdf = [path.split(".")[0].split('/')[-1] for path in list_sdf_after]
# print(name_sdf)
print(len(name_sdf))
list_pdb = glob.glob("/Users/duckhoan/Documents/VS_Code/ViDok/Auto_download_vidok/vidok_file_1/*.pdbqt")
name_pdb = [path.split(".")[0].split('/')[-1] for path in list_pdb]
# print(name_pdb)
print(len(name_pdb))

# for i in name_pdb:
#     if i in name_sdf:
#         shutil.copy("/Users/duckhoan/Documents/VS_Code/ViDok/Auto_download_vidok/vidok_file_1/"+i+".pdbqt", '/Users/duckhoan/Documents/VS_Code/ViDok/Auto_download_vidok/ligand_after_format/')

print(len(glob.glob('/Users/duckhoan/Documents/VS_Code/ViDok/Auto_download_vidok/ligand_after_format/*')))
# for i in name_pdb:
#     if i not in name_sdf:
#         link = i+".pdbqt"
#         if os.path.exists(link):
#             os.remove(link)
# print(len(list_file))
# count = 0
# for i in list_sdf_after:
#         ligands_sdf = SDMolSupplier(i)
#         ligand = ligands_sdf[0]
#         if ligand != None:
#                 count+=1
#         else:
#             print(i)
#                 # if os.path.exists(i):
#                 #         os.remove(i)

# print(count)

0
983
0


In [15]:
ligand

NameError: name 'ligand' is not defined

In [ ]:

receptors_sdf = SDMolSupplier("/Users/duckhoan/Documents/VS_Code/ViDok/PROTEIN/ZINC000085862539_982.sdf")
receptor = receptors_sdf[0]
receptor != None

In [ ]:
receptor
# MolToImage(receptor, size=(500, 500))

In [ ]:
print(NumHAcceptors(ligand))
print(NumHDonors(receptor))

In [ ]:
print(NumHAcceptors(receptor))
print(NumHDonors(ligand))

In [ ]:
class _ChemicalFeaturesFactory:
    """This is a singleton class for RDKit base features."""
    _instance = None

    @classmethod
    def get_instance(cls):
        try:
            from rdkit import RDConfig
            from rdkit.Chem import ChemicalFeatures
        except ModuleNotFoundError:
            raise ImportError("This class requires RDKit to be installed.")

        if not cls._instance:
            fdefName = os.path.join(RDConfig.RDDataDir, 'BaseFeatures.fdef')
            cls._instance = ChemicalFeatures.BuildFeatureFactory(fdefName)
        return cls._instance

In [ ]:
def construct_hydrogen_bonding_info(mol):
    """Construct hydrogen bonding infos about a molecule.
    Parameters
    ---------
    mol: rdkit.Chem.rdchem.Mol
    RDKit mol object
    Returns
    -------
    List[Tuple[int, str]]
    A list of tuple `(atom_index, hydrogen_bonding_type)`.
    The `hydrogen_bonding_type` value is "Acceptor" or "Donor".
    """
    factory = _ChemicalFeaturesFactory.get_instance()
    feats = factory.GetFeaturesForMol(mol)
    hydrogen_bonding = []
    for f in feats:
        hydrogen_bonding.append((f.GetAtomIds()[0], f.GetFamily()))
    return hydrogen_bonding

In [ ]:
ligand_bonding_info = construct_hydrogen_bonding_info(ligand)
print(ligand_bonding_info)

In [ ]:
receptor_bonding_info = construct_hydrogen_bonding_info(receptor)
# list(filter(lambda x: 1800>=x[0]>=1798, receptor_bonding_info))

# with open("receptor_hydrogen_bonding.txt", "w") as f:
#     f.write(str(receptor_bonding_info))

In [ ]:
for tup in receptor_bonding_info:
    if tup[0]==1811:
        print(tup[1])

In [ ]:
def read_candidate_link_results(res_file):
    list_bonds = []
    with open(res_file, "r", encoding="utf-8") as rf:
        lines = rf.read().split("\n")
        for line in lines:
            if line != "":
                list_idx = line.split(",")
                list_idx = [int(x) for x in list_idx if x != ""]
                if len(list_idx) < 2:
                    continue
                for r_idx in list_idx[1:]:
                    list_bonds.append((list_idx[0], r_idx))
                    
    return list_bonds
        
list_candidate_bonds = read_candidate_link_results("results/nearest_atoms/lr_mapping/Ligand_6lu7.pdb-Receptor_ViDok.pdb.csv")
print(list_candidate_bonds)

In [ ]:
list_candidate_bonds = [(8, 1586), (8, 1587), (8, 1594), (8, 1593), (8, 1597), (8, 1599), (8, 1598), (8, 1814), (8, 1811), (8, 1805), (8, 1807), (8, 1810), (8, 1804), (8, 1798), (8, 1799), (8, 1806), (8, 1815), (8, 1578), (8, 1821), (8, 1785), (8, 1770), (8, 1786), (8, 1796), (8, 1797), (8, 1787), (8, 1795), (9, 1586), (9, 1587), (9, 1593), (9, 1599), (9, 1598), (9, 1811), (9, 1810), (9, 1798), (9, 1799), (9, 1815), (9, 1592), (9, 1584), (9, 1577), (9, 1578), (9, 1572), (9, 1575), (9, 1821), (9, 1819), (9, 1785), (9, 1770), (9, 1786), (9, 1796), (9, 1795), (10, 1586), (10, 1599), (10, 1799), (10, 1577), (10, 1570), (10, 1578), (10, 1574), (10, 1572), (10, 1575), (10, 1791), (10, 1821), (10, 1789), (10, 1785), (10, 1788), (10, 1792), (10, 1790), (10, 1770), (10, 1786), (10, 1783), (10, 1796), (10, 1795), (11, 1798), (11, 1799), (11, 1578), (11, 1791), (11, 1794), (11, 1821), (11, 1789), (11, 1785), (11, 1788), (11, 1792), (11, 1793), (11, 1790), (11, 1770), (11, 1769), (11, 1786), (11, 1783), (11, 1796), (11, 1784), (11, 1797), (11, 1787), (11, 1795), (12, 1586), (12, 1593), (12, 1599), (12, 1598), (12, 1814), (12, 1811), (12, 1810), (12, 1804), (12, 1798), (12, 1799), (12, 1812), (12, 1816), (12, 1815), (12, 1592), (12, 1573), (12, 1818), (12, 1817), (12, 1571), (12, 1584), (12, 1577), (12, 1570), (12, 1578), (12, 1572), (12, 1575), (12, 1767), (12, 1821), (12, 1820), (12, 1819), (12, 1785), (12, 1770), (12, 1769), (12, 1786), (12, 1783), (12, 1796), (12, 1797), (12, 1795), (13, 1586), (13, 1579), (13, 1576), (13, 1584), (13, 1577), (13, 1570), (13, 1578), (13, 1574), (13, 1567), (13, 1572), (13, 1575), (13, 1568), (13, 1791), (13, 1789), (13, 1785), (13, 1788), (13, 1790), (14, 1579), (14, 1576), (14, 1577), (14, 1570), (14, 1578), (14, 1574), (14, 1575), (14, 1791), (14, 1794), (14, 1789), (14, 1785), (14, 1788), (14, 1792), (14, 1790), (15, 1579), (15, 1576), (15, 1571), (15, 1577), (15, 1570), (15, 1578), (15, 1574), (15, 1567), (15, 1572), (15, 1575), (15, 1568), (15, 1791), (15, 1789), (15, 1792), (15, 1790), (15, 1556), (16, 1579), (16, 1576), (16, 1569), (16, 1571), (16, 1577), (16, 1570), (16, 1578), (16, 1574), (16, 1567), (16, 1572), (16, 1575), (16, 1568), (16, 1791), (16, 1565), (16, 1555), (16, 1556), (17, 1579), (17, 1576), (17, 1577), (17, 1578), (17, 1574), (17, 1575), (17, 1791), (17, 1794), (17, 1792), (17, 1790), (18, 1575), (18, 1791), (18, 1794), (18, 1792), (18, 1790), (18, 1360), (19, 1578), (19, 1575), (19, 1791), (19, 1794), (19, 1792), (19, 1790), (20, 1571), (20, 1570), (20, 1578), (20, 1574), (20, 1567), (20, 1575), (20, 1791), (20, 1794), (20, 1789), (20, 1785), (20, 1788), (20, 1792), (20, 1790), (20, 1556), (21, 1571), (21, 1570), (21, 1574), (21, 1567), (21, 1575), (21, 1568), (21, 1791), (21, 1789), (21, 1790), (21, 1383), (21, 1565), (21, 1555), (21, 383), (21, 382), (21, 1556), (21, 381), (21, 378), (22, 1570), (22, 1574), (22, 1567), (22, 1575), (22, 1568), (22, 1791), (22, 1383), (22, 1565), (22, 1555), (22, 383), (22, 382), (22, 1556), (22, 381), (22, 378), (23, 1570), (23, 1567), (23, 1575), (23, 1791), (23, 1790), (23, 1360), (23, 1365), (23, 1383), (23, 1556), (24, 1570), (24, 1567), (24, 1791), (24, 1789), (24, 1790), (24, 383), (24, 382), (24, 1556), (24, 381), (24, 378), (24, 379), (24, 380), (24, 377), (24, 376), (24, 452), (24, 451), (24, 450), (24, 449), (25, 1570), (25, 1791), (25, 1767), (25, 1789), (25, 1785), (25, 1788), (25, 1790), (25, 1770), (25, 1769), (25, 1783), (25, 1784), (25, 1762), (25, 1759), (25, 383), (25, 382), (25, 1556), (25, 381), (25, 378), (25, 379), (25, 380), (25, 377), (25, 376), (25, 1766), (25, 504), (25, 1760), (25, 1761), (25, 503), (25, 1768), (25, 452), (25, 451), (25, 450), (25, 449), (25, 448), (26, 1791), (26, 1767), (26, 1789), (26, 1790), (26, 1769), (26, 1783), (26, 1784), (26, 1762), (26, 1759), (26, 383), (26, 382), (26, 381), (26, 378), (26, 379), (26, 380), (26, 377), (26, 373), (26, 376), (26, 375), (26, 1766), (26, 504), (26, 1760), (26, 1761), (26, 503), (26, 1768), (26, 452), (26, 451), (26, 450), (26, 449), (26, 502), (26, 448), (26, 446), (27, 1753), (27, 1571), (27, 1570), (27, 1567), (27, 1572), (27, 1791), (27, 1767), (27, 1789), (27, 1770), (27, 1769), (27, 1783), (27, 1784), (27, 1762), (27, 1757), (27, 1759), (27, 1565), (27, 1555), (27, 383), (27, 382), (27, 1556), (27, 381), (27, 378), (27, 380), (27, 377), (27, 376), (27, 1766), (27, 504), (27, 1760), (27, 1761), (27, 503), (27, 1768), (0, 1594), (0, 1593), (0, 1597), (0, 1599), (0, 1598), (0, 1811), (0, 1809), (0, 1804), (0, 1799), (0, 1806), (1, 1597), (1, 1598), (1, 1805), (1, 1809), (1, 1804), (1, 1806), (2, 1598), (2, 1809), (3, 1598), (3, 1811), (3, 1805), (3, 1807), (3, 1809), (3, 1804), (3, 1798), (3, 1799), (3, 1806), (3, 1797), (4, 1598), (4, 1811), (4, 1805), (4, 1807), (4, 1809), (4, 1810), (4, 1804), (4, 1798), (4, 1799), (4, 1806), (4, 1786), (4, 1797), (4, 1787), (4, 1795), (5, 1594), (5, 1593), (5, 1597), (5, 1599), (5, 1598), (5, 1811), (5, 1805), (5, 1807), (5, 1809), (5, 1810), (5, 1804), (5, 1798), (5, 1799), (5, 1806), (5, 1797), (6, 1587), (6, 1594), (6, 1593), (6, 1597), (6, 1599), (6, 1598), (6, 1811), (6, 1810), (6, 1804), (6, 1798), (6, 1799), (6, 1806), (6, 1578), (6, 1795), (7, 1588), (7, 1586), (7, 1587), (7, 1594), (7, 1593), (7, 1597), (7, 1599), (7, 1598), (7, 1811), (7, 1799), (7, 1578), (28, 1567), (28, 1575), (28, 1550), (28, 1548), (28, 1551), (28, 1360), (28, 1369), (28, 1348), (28, 1370), (28, 1355), (28, 1364), (28, 1356), (28, 1375), (28, 1347), (28, 1365), (28, 1376), (28, 1378), (28, 1377), (28, 1372), (28, 1379), (28, 1382), (28, 1383), (28, 1565), (28, 1555), (28, 1556), (28, 381), (29, 1550), (29, 1548), (29, 1380), (29, 1551), (29, 1371), (29, 1369), (29, 1348), (29, 1370), (29, 1374), (29, 1364), (29, 1375), (29, 1365), (29, 1376), (29, 1378), (29, 1367), (29, 1377), (29, 1373), (29, 1366), (29, 1372), (29, 1379), (29, 1381), (29, 1554), (29, 1543), (29, 1382), (29, 1383), (29, 1565), (29, 1555), (29, 383), (29, 382), (29, 1556), (29, 381), (29, 378), (30, 1380), (30, 1371), (30, 1369), (30, 1348), (30, 1370), (30, 1355), (30, 1364), (30, 1356), (30, 1375), (30, 1365), (30, 1378), (30, 1368), (30, 1367), (30, 1377), (30, 1366), (30, 1372), (30, 1379), (30, 1381), (30, 1382), (30, 1383), (30, 238), (30, 383), (30, 382), (30, 1556), (30, 381), (30, 378), (30, 241), (30, 379), (31, 1369), (31, 1370), (31, 1355), (31, 1364), (31, 1356), (31, 1365), (31, 1358), (31, 1378), (31, 1367), (31, 1377), (31, 1366), (31, 1379), (31, 1382), (31, 1383), (31, 238), (31, 381), (31, 378), (31, 241), (31, 379), (32, 1574), (32, 1567), (32, 1575), (32, 1568), (32, 1549), (32, 1546), (32, 1550), (32, 1545), (32, 1548), (32, 1551), (32, 1337), (32, 1360), (32, 1371), (32, 1369), (32, 1348), (32, 1370), (32, 1374), (32, 1355), (32, 1364), (32, 1375), (32, 1347), (32, 1365), (32, 1353), (32, 1376), (32, 1378), (32, 1377), (32, 1372), (32, 1379), (32, 1566), (32, 1382), (32, 1383), (32, 1565), (32, 1555), (32, 1556), (33, 1335), (33, 1336), (33, 1583), (33, 1579), (33, 1576), (33, 1569), (33, 1574), (33, 1567), (33, 1575), (33, 1568), (33, 1549), (33, 1550), (33, 1548), (33, 1344), (33, 1551), (33, 1337), (33, 1360), (33, 1346), (33, 1348), (33, 1370), (33, 1355), (33, 1375), (33, 1347), (33, 1365), (33, 1353), (33, 1376), (33, 1383), (33, 1565), (33, 1555), (33, 1556), (34, 1335), (34, 1336), (34, 1579), (34, 1575), (34, 1550), (34, 1344), (34, 1551), (34, 1360), (34, 1346), (34, 1348), (34, 1370), (34, 1355), (34, 1364), (34, 1356), (34, 1375), (34, 1347), (34, 1354), (34, 1365), (34, 1359), (34, 1353), (34, 1358), (34, 1376), (34, 1361), (35, 1335), (35, 1336), (35, 1634), (35, 1583), (35, 1581), (35, 1582), (35, 1579), (35, 1580), (35, 1575), (35, 1344), (35, 1551), (35, 1360), (35, 1346), (35, 1348), (35, 1355), (35, 1347), (35, 1354), (35, 1359), (35, 1353), (35, 1358), (36, 1332), (36, 1333), (36, 1335), (36, 1336), (36, 1634), (36, 1633), (36, 1631), (36, 1583), (36, 1581), (36, 1582), (36, 1579), (36, 1580), (36, 1576), (36, 1574), (36, 1575), (36, 1550), (36, 1334), (36, 1344), (36, 1345), (36, 1551), (36, 1337), (36, 1360), (36, 1346), (36, 1348), (36, 1355), (36, 1375), (36, 1347), (36, 1354), (36, 1353), (36, 1376), (37, 1333), (37, 1335), (37, 1336), (37, 1634), (37, 1633), (37, 1631), (37, 1583), (37, 1581), (37, 1582), (37, 1579), (37, 1580), (37, 1576), (37, 1569), (37, 1574), (37, 1567), (37, 1575), (37, 1568), (37, 1549), (37, 1338), (37, 1550), (37, 1548), (37, 1334), (37, 1344), (37, 1345), (37, 1551), (37, 1337), (37, 1360), (37, 1346), (37, 1348), (37, 1375), (37, 1347), (37, 1353), (37, 1376), (37, 1565), (38, 1332), (38, 1333), (38, 1335), (38, 1336), (38, 1340), (38, 1632), (38, 1634), (38, 1629), (38, 1633), (38, 1631), (38, 1583), (38, 1628), (38, 1627), (38, 1581), (38, 1579), (38, 1580), (38, 1576), (38, 1569), (38, 1574), (38, 1567), (38, 1575), (38, 1568), (38, 1549), (38, 1338), (38, 1547), (38, 1546), (38, 1550), (38, 1545), (38, 1548), (38, 1339), (38, 1334), (38, 1344), (38, 1551), (38, 1337), (38, 1346), (38, 1348), (38, 1375), (38, 1347), (38, 1376), (38, 1566), (38, 1565), (39, 1570), (39, 1574), (39, 1567), (39, 1575), (39, 1568), (39, 1550), (39, 1548), (39, 1551), (39, 1365), (39, 1378), (39, 1566), (39, 1554), (39, 1382), (39, 1383), (39, 1565), (39, 1555), (39, 383), (39, 382), (39, 1556), (39, 381), (40, 1364), (40, 1365), (40, 1378), (40, 1366), (40, 1382), (40, 1383), (40, 381), (40, 378), (40, 241), (40, 379), (40, 452), (41, 1360), (41, 1363), (41, 1355), (41, 1364), (41, 1356), (41, 1365), (41, 1359), (41, 1358), (41, 1366), (41, 1361), (41, 1362), (42, 1360), (42, 1369), (42, 1348), (42, 1370), (42, 1355), (42, 1364), (42, 1356), (42, 1365), (42, 1359), (42, 1353), (42, 1357), (42, 1358), (42, 1378), (42, 1368), (42, 1367), (42, 1377), (42, 1366), (42, 1382), (42, 1383), (42, 1361), (42, 228), (43, 1363), (43, 1364), (43, 1365), (43, 1358), (43, 223), (43, 225), (44, 232), (44, 228), (44, 223), (44, 220), (44, 210), (44, 217), (44, 224), (44, 225), (44, 209), (44, 218), (45, 232), (45, 228), (45, 227), (45, 223), (45, 226), (45, 220), (45, 210), (45, 217), (45, 224), (45, 225), (45, 209), (45, 215), (45, 218), (45, 221), (45, 219), (45, 229), (46, 1365), (46, 1366), (46, 235), (46, 233), (46, 238), (46, 232), (46, 228), (46, 227), (46, 223), (46, 226), (46, 220), (46, 210), (46, 217), (46, 224), (46, 225), (46, 209), (46, 215), (46, 218), (46, 221), (46, 219), (46, 229), (47, 1364), (47, 1365), (47, 1366), (47, 233), (47, 238), (47, 232), (47, 228), (47, 227), (47, 223), (47, 226), (47, 220), (47, 217), (47, 224), (47, 225), (47, 218), (48, 1355), (48, 1364), (48, 1356), (48, 1365), (48, 1358), (48, 1366), (48, 228), (48, 223), (48, 225)]

In [ ]:
def check_hydrogen_bonds(list_candidate_bonds, ligand_bonding_info, receptor_bonding_info):
    ligand_aod = {}
    receptor_aod = {}
    hydrogen_bonds = []
    
    ligand_aod["Acceptor"] = list(y[0] for y in filter(lambda x: x[1] == "Acceptor", ligand_bonding_info))
    ligand_aod["Donor"] = list(y[0] for y in filter(lambda x: x[1] == "Donor", ligand_bonding_info))
    
    receptor_aod["Acceptor"] = list(y[0] for y in filter(lambda x: x[1] == "Acceptor", receptor_bonding_info))
    receptor_aod["Donor"] = list(y[0] for y in filter(lambda x: x[1] == "Donor", receptor_bonding_info))
    
    for latm_idx, ratm_idx in list_candidate_bonds:
        if latm_idx in ligand_aod["Acceptor"] and ratm_idx in receptor_aod["Donor"]:
            hydrogen_bonds.append((latm_idx, ratm_idx))
            
        if latm_idx in ligand_aod["Donor"] and ratm_idx in receptor_aod["Acceptor"]:
            hydrogen_bonds.append((latm_idx, ratm_idx))
            
    return hydrogen_bonds

list_hydrogen_bonds = check_hydrogen_bonds(list_candidate_bonds, ligand_bonding_info, receptor_bonding_info)
print(list_hydrogen_bonds)